## Dataset Download

In [ ]:
!gdown 1doU9FE1tJ-0IL4tKZUOOp76JkXKuTkFU

Downloading...
From (original): https://drive.google.com/uc?id=1doU9FE1tJ-0IL4tKZUOOp76JkXKuTkFU
From (redirected): https://drive.google.com/uc?id=1doU9FE1tJ-0IL4tKZUOOp76JkXKuTkFU&confirm=t&uuid=5e3bef8b-d79b-4ea5-810e-5ce7a67ce72a
To: /kaggle/working/anxiety_dataset_complete.zip
100%|██████████████████████████████████████| 1.41G/1.41G [00:17<00:00, 80.8MB/s]


In [ ]:
!unzip anxiety_dataset_complete.zip > /dev/null

In [40]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import CLIPImageProcessor, CLIPVisionModel
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image
import json
import math

TRAIN_RATIO = 0.7  # 70% train
VAL_RATIO = 0.1  # 10% validation
TEST_RATIO = 0.2  # 20% test
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
LR = 2e-5
FUSION_DIM = 768
LABELS = ["Nervousness", "Lack of Worry Control", "Excessive Worry",
          "Difficulty Relaxing", "Restlessness", "Impending Doom"]
LABEL_MAP = {i: idx for idx, i in enumerate(LABELS)}

## Custom Dataset

In [41]:
class ImageOnlyAnxietyDataset(Dataset):
    def __init__(self, data, image_path, image_processor):
        self.data = data
        self.image_processor = image_processor
        self.img_path = image_path
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]

        image_path = os.path.join(self.img_path, sample["sample_id"] + ".jpg")
        image = Image.open(image_path).convert("RGB")
        image_tensor = self.image_processor(image, return_tensors="pt")

        label = LABEL_MAP[sample["meme_anxiety_category"]]

        return {
            "image": image_tensor,
            "label": torch.tensor(label, dtype=torch.long)
        }

def custom_collate_fn(batch):
    labels = torch.stack([item['label'] for item in batch])

    images = {}
    for key in batch[0]['image'].keys():
        if isinstance(batch[0]['image'][key], torch.Tensor):
            images[key] = torch.stack([item['image'][key].squeeze(0) for item in batch])

    return {
        'image': images,
        'label': labels,
    }

## Model Definition

In [42]:
class ImageOnlyModel(nn.Module):
    def __init__(self, num_classes=6):
        super(ImageOnlyModel, self).__init__()

        self.vision_encoder = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
        self.vision_dim = self.vision_encoder.config.hidden_size

        self.vision_projections = nn.ModuleList([
            nn.Linear(self.vision_dim, 1024) for _ in range(3)
        ])

        self.expert_nets = nn.ModuleList([
            nn.Sequential(
                nn.Linear(1024, 1024),
                nn.LayerNorm(1024),
                nn.GELU(),
                nn.Dropout(0.2)
            ) for _ in range(4)
        ])

        self.moe_gate = nn.Linear(1024, 4)

        self.classifier = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.LayerNorm(1024),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(1024, num_classes)
        )


    def forward(self, image_features):
        vision_outputs = self.vision_encoder(**image_features, output_hidden_states=True)
        vision_features = [
            vision_outputs.hidden_states[-i][:, 0] for i in range(1, 4)
        ]
        vision_features = [
            proj(feat) for proj, feat in zip(self.vision_projections, vision_features)
        ]

        combined_feature = sum(vision_features) / len(vision_features)

        expert_outputs = [expert(combined_feature) for expert in self.expert_nets]
        expert_gates = F.softmax(self.moe_gate(combined_feature), dim=1)

        moe_output = torch.zeros_like(expert_outputs[0])
        for i, expert_out in enumerate(expert_outputs):
            moe_output += expert_out * expert_gates[:, i].unsqueeze(1)

        logits = self.classifier(moe_output)

        return logits

## Training Functions

In [43]:
def train_image_only_model(model, train_data, val_data, img_path, epochs, model_save_name):
    image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

    train_dataset = ImageOnlyAnxietyDataset(train_data, img_path, image_processor)
    val_dataset = ImageOnlyAnxietyDataset(val_data, img_path, image_processor)

    print("Train Set Size:", len(train_dataset))
    print("Validation Set Size:", len(val_dataset))

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=custom_collate_fn
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=custom_collate_fn
    )

    optimizer = optim.AdamW(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2, verbose=True
    )
    criterion = nn.CrossEntropyLoss()
    
    model = model.to(DEVICE)
    model = nn.DataParallel(model)

    best_f1 = 0
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        model.train()
        train_loss = 0
        train_preds, train_labels = [], []

        for batch in tqdm(train_loader, desc="Training"):
            labels = batch["label"].to(DEVICE)
            image_features = {k: v.to(DEVICE) for k, v in batch["image"].items()}

            optimizer.zero_grad()
            logits = model(image_features)
            loss = criterion(logits, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss += loss.item()
            predictions = torch.argmax(logits, dim=1).cpu().numpy()
            train_preds.extend(predictions)
            train_labels.extend(labels.cpu().numpy())

        train_loss = train_loss / len(train_loader)
        train_macro_f1 = f1_score(train_labels, train_preds, average="macro")
        train_weighted_f1 = f1_score(train_labels, train_preds, average="weighted")

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Train Macro-F1: {train_macro_f1:.4f}, Weighted-F1: {train_weighted_f1:.4f}")

        val_loss, val_macro_f1, val_weighted_f1 = evaluate_image_model(
            model, val_loader, criterion
        )

        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Macro-F1: {val_macro_f1:.4f}, Weighted-F1: {val_weighted_f1:.4f}")

        scheduler.step(val_macro_f1)

        f1_hm = 2 * val_macro_f1 * val_weighted_f1 / (val_macro_f1 + val_weighted_f1)
        if f1_hm > best_f1:
            best_f1 = f1_hm
            torch.save(model.state_dict(), f"{model_save_name}_anxiety.pth")
            print("Best model saved!")

    return model

def evaluate_image_model(model, loader, criterion):
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            labels = batch["label"].to(DEVICE)
            image_features = {k: v.to(DEVICE) for k, v in batch["image"].items()}

            logits = model(image_features)
            loss = criterion(logits, labels)

            val_loss += loss.item()
            predictions = torch.argmax(logits, dim=1).cpu().numpy()
            val_preds.extend(predictions)
            val_labels.extend(labels.cpu().numpy())

    val_loss = val_loss / len(loader)
    val_macro_f1 = f1_score(val_labels, val_preds, average="macro")
    val_weighted_f1 = f1_score(val_labels, val_preds, average="weighted")

    target_names = [k for k, v in sorted(LABEL_MAP.items(), key=lambda x: x[1])]
    report = classification_report(val_labels, val_preds, target_names=target_names)
    print(report)

    return val_loss, val_macro_f1, val_weighted_f1

## Model Training

In [44]:
full_train_data = json.load(open("anxiety_train_llava_dataset.json", "r"))
test_data = json.load(open("anxiety_test_llava_dataset.json", "r"))

labels = [LABEL_MAP[item["meme_anxiety_category"]] for item in full_train_data]

train_size = math.ceil(len(full_train_data) * TRAIN_RATIO / (TRAIN_RATIO + VAL_RATIO))
train_data, val_data = train_test_split(
    full_train_data, train_size=train_size, stratify=labels, random_state=42
)

img_path = "anxiety_train_image"

image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

test_dataset = ImageOnlyAnxietyDataset(test_data, "anxiety_test_image", image_processor)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=custom_collate_fn
)

In [ ]:
model = ImageOnlyModel(num_classes=len(LABEL_MAP))

trained_model, image_processor = train_image_only_model(
    model,
    train_data,
    val_data,
    img_path,
    epochs=30,
    model_save_name="only_image"
)

Train Set Size: 2153
Validation Set Size: 307

Epoch 1/30


Training: 100%|██████████| 135/135 [01:19<00:00,  1.69it/s]


Train Loss: 1.8536
Train Macro-F1: 0.2953, Weighted-F1: 0.2953


Evaluating: 100%|██████████| 20/20 [00:09<00:00,  2.21it/s]


                       precision    recall  f1-score   support

          Nervousness       0.67      0.04      0.07        53
Lack of Worry Control       0.71      0.32      0.44        47
      Excessive Worry       0.00      0.00      0.00        46
  Difficulty Relaxing       0.33      0.88      0.48        51
         Restlessness       0.24      0.26      0.25        58
       Impending Doom       0.32      0.50      0.39        52

             accuracy                           0.34       307
            macro avg       0.38      0.33      0.27       307
         weighted avg       0.38      0.34      0.27       307

Validation Loss: 1.6980
Validation Macro-F1: 0.2708, Weighted-F1: 0.2716
Best model saved!

Epoch 2/30


...

                       precision    recall  f1-score   support

          Nervousness       0.47      0.43      0.45        53
Lack of Worry Control       0.58      0.53      0.56        47
      Excessive Worry       0.53      0.37      0.44        46
  Difficulty Relaxing       0.82      0.53      0.64        51
         Restlessness       0.29      0.47      0.36        58
       Impending Doom       0.49      0.54      0.51        52

             accuracy                           0.48       307
            macro avg       0.53      0.48      0.49       307
         weighted avg       0.52      0.48      0.49       307

Validation Loss: 3.4880
Validation Macro-F1: 0.4928, Weighted-F1: 0.4896

Epoch 30/30


Training: 100%|██████████| 135/135 [01:18<00:00,  1.73it/s]


Train Loss: 0.3474
Train Macro-F1: 0.9966, Weighted-F1: 0.9968


Evaluating: 100%|██████████| 20/20 [00:08<00:00,  2.26it/s]


                       precision    recall  f1-score   support

          Nervousness       0.47      0.43      0.45        53
Lack of Worry Control       0.57      0.53      0.55        47
      Excessive Worry       0.53      0.37      0.44        46
  Difficulty Relaxing       0.82      0.53      0.64        51
         Restlessness       0.30      0.48      0.37        58
       Impending Doom       0.49      0.52      0.50        52

             accuracy                           0.48       307
            macro avg       0.53      0.48      0.49       307
         weighted avg       0.52      0.48      0.49       307

Validation Loss: 3.4840
Validation Macro-F1: 0.4920, Weighted-F1: 0.4892


## Inference

In [71]:
!gdown 1rdLwO2mZTYoD7NtMYjpZW_owsPzyHjvi

Downloading...
From (original): https://drive.google.com/uc?id=1rdLwO2mZTYoD7NtMYjpZW_owsPzyHjvi
From (redirected): https://drive.google.com/uc?id=1rdLwO2mZTYoD7NtMYjpZW_owsPzyHjvi&confirm=t&uuid=e3037a52-acd3-480e-9130-43d9e05bc16b
To: /kaggle/working/only_image_anxiety.pth
100%|█████████████████████████████████████████| 380M/380M [00:02<00:00, 151MB/s]


In [72]:
def inference(test_data, model_path):
    model = ImageOnlyModel(num_classes=len(LABEL_MAP))

    model = model.to(DEVICE)

    weights = torch.load(model_path, map_location=DEVICE, weights_only=True)
    weights_single = {k.replace("module.", ""): v for k, v in weights.items() if "contrastive_projection" not in k}

    model.load_state_dict(weights_single)
    image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

    test_dataset = ImageOnlyAnxietyDataset(test_data, "anxiety_test_image", image_processor)
    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=custom_collate_fn
    )

    loss, macro_f1, weighted_f1 = evaluate_image_model(
        model, test_loader, nn.CrossEntropyLoss()
    )

    print(f"Test Loss: {loss:.4f}")
    print(f"Test Macro-F1: {macro_f1:.4f}, Weighted-F1: {weighted_f1:.4f}")

    return macro_f1, weighted_f1

In [73]:
inference(test_data, "only_image_anxiety.pth")

Evaluating: 100%|██████████| 39/39 [00:17<00:00,  2.23it/s]


                       precision    recall  f1-score   support

          Nervousness       0.44      0.36      0.40       106
Lack of Worry Control       0.67      0.39      0.50        94
      Excessive Worry       0.41      0.39      0.40        92
  Difficulty Relaxing       0.76      0.58      0.66       102
         Restlessness       0.37      0.74      0.49       116
       Impending Doom       0.55      0.40      0.46       105

             accuracy                           0.48       615
            macro avg       0.53      0.48      0.48       615
         weighted avg       0.53      0.48      0.49       615

Test Loss: 3.2406
Test Macro-F1: 0.4845, Weighted-F1: 0.4852


(0.48453214917837184, 0.48522518139802373)